In [ ]:
# Step 1: Install required libraries
!pip install -q transformers accelerate peft bitsandbytes datasets

# Step 2: Authenticate with Hugging Face
from huggingface_hub import login
login("your-access-token")

# Step 3: Import necessary components
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
    TrainerCallback  # Import TrainerCallback here
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import time

# Step 4: Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token

# Step 5: Prepare dataset
def chunk_text(text, chunk_size=512):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

with open("/kaggle/input/harry-potter/HarryPotter4Books.txt", "r") as f:
    text = f.read()

chunks = chunk_text(text)
dataset = Dataset.from_dict({"text": chunks})

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Step 6: Configure model with PEFT
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Step 7: Create custom callback for periodic generation
class StoryGeneratorCallback(TrainerCallback):  # Now TrainerCallback is defined
    def __init__(self, tokenizer, interval=300):  # 300 seconds = 5 minutes
        self.tokenizer = tokenizer
        self.interval = interval
        self.last_generation = time.time()
        self.prompts = [
            "Harry Potter and Hermione Granger were exploring",
            "Suddenly, the Dark Mark appeared",
            "In the Great Hall, Dumbledore announced",
            "Ron Weasley dropped his wand when",
            "A mysterious package arrived at Hogwarts containing"
        ]
    
    def on_step_end(self, args, state, control, **kwargs):
        if time.time() - self.last_generation > self.interval:
            self.last_generation = time.time()
            model = kwargs["model"]
            prompt = self.prompts[state.global_step % len(self.prompts)]
            
            # Generate story
            inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=150,
                    temperature=0.7,
                    top_p=0.9,
                    do_sample=True
                )
            
            story = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"\n\n--- Generated Story at Step {state.global_step} ---")
            print(story)
            print("\n" + "="*50 + "\n")
        return control

# Step 8: Set up training configuration
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./hp_storyteller",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.001,
    warmup_ratio=0.05,
    fp16=True,
    logging_steps=50,
    optim="paged_adamw_8bit",
    report_to="none",
    save_strategy="steps",
    save_steps=500
)

# Step 9: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    callbacks=[StoryGeneratorCallback(tokenizer)]
)

# Step 10: Start training
print("Starting training...")
trainer.train()

# Step 11: Save the final model
model.save_pretrained("./hp_storyteller_final")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.3 MB/s eta 0:00:00:00:0100:01


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622
Starting training...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
50,10.668600
100,9.148300
150,9.021200
200,8.886900
250,8.834100
300,8.771000
350,8.729800
400,8.708300
450,8.686000
500,8.538900




--- Generated Story at Step 60 ---
Harry Potter and Hermione Granger were exploring the forest, looking for the entrance to the Chamber. Harry was still dressed in his school robes, but Hermione was wearing a long, billowy cloak to keep warm.
"I can't believe we're doing this," Harry said, his voice hushed. "We're not supposed to be in the forest. I don't want to get in trouble."
"We have to find the Chamber," Hermione said firmly. "We have to get the Philosopher's Stone."
"But it's dangerous," Harry said. "I don't want to get caught. We'll be expelled."
"We have to do it," Hermione said. "




--- Generated Story at Step 115 ---
Harry Potter and Hermione Granger were exploring the
 Harry Potter and the Goblet of Fire 31
corridors. They were in the process of
discovering that the giant, filthy, and very
angry giant, Rubeus Hagrid, was an
unlikely friend of theirs.

"Dumbledore said I could come and get
you," Hagrid was saying to Harry and
Hermione, his face twisted in a grimace of
de

In [6]:
def generate_story(prompt, max_new_tokens=210, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True
        )
    
    story = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return story


In [7]:
print(generate_story("harry was running along a river"))
print("\n"*20)
print(generate_story("once there lived and evil wizard sam"))
print("\n"*20)
print(generate_story("hermoine found ron near a well fighting a monster"))

harry was running along a riverbank, and the
cottage was in sight.

"Come on, Harry, quick!"

"What's going on?"

"You don't know? You must have been dreaming!"

"Dreaming? What about the dementor?"

"The dementor?"

"I was at the Quidditch World Cup, and it came out of nowhere -"

"No, Harry, you weren't - you were in bed in Gryffindor Tower, and
you'd just woken up -"

"I know I was in bed! But I must've been dreaming, because -"

"No, you weren't dreaming!"

"How can you tell? What's going on?"

"You'll see in a minute! Come on!"

Harry had a feeling that his head was going to burst with questions,
but he





















once there lived and evil wizard sammeth
sorcerer. He used to torture and kill people here. And he left behind
something of his. A spellbook. I have it with me. And I have been
trying to learn it. I've been trying to become the new Lord Voldemort.

"But I couldn't. I didn't understand the spells. I couldn't even do the
 simplest thing - turn myself into an anim

In [8]:
merged_model = model.merge_and_unload()

# Save full model
merged_model.save_pretrained("./hp_full_model")
tokenizer.save_pretrained("./hp_full_model")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('./hp_full_model/tokenizer_config.json',
 './hp_full_model/special_tokens_map.json',
 './hp_full_model/tokenizer.model',
 './hp_full_model/added_tokens.json',
 './hp_full_model/tokenizer.json')